In [2]:
import geopandas as gpd
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:

### Sampleo "Pesado" de Censo, para poder tener buena calidad en areas pequenas.

### Prediccion en base a modelos. Ultimos 4 trimestres disponibles.

### Generar dataset de pobreza


In [4]:

from datetime import datetime
hoy = datetime.today().strftime('%Y-%m-%d')
hoy

'2022-09-14'

# 1. Sampleo Censo

In [5]:
import datetime as dt
## Trimestres con ingresos disponibles (depende de disponibilidad de microdatos EPH)
import glob

fuente_modelos = 'encuestador-de-hogares' # 
modelos_trimestrales = '/media/miglesia/Elements/suite/'+fuente_modelos+'/fitted_RF/clf4_' # use your path

allFiles = []
allFiles += glob.glob(modelos_trimestrales +'*'+'ARG')
allFiles = sorted(allFiles)
FilesActual = allFiles[-4:]
FilesActual

['/media/miglesia/Elements/suite/encuestador-de-hogares/fitted_RF/clf4_2021-05-15_ARG',
 '/media/miglesia/Elements/suite/encuestador-de-hogares/fitted_RF/clf4_2021-08-15_ARG',
 '/media/miglesia/Elements/suite/encuestador-de-hogares/fitted_RF/clf4_2021-11-15_ARG',
 '/media/miglesia/Elements/suite/encuestador-de-hogares/fitted_RF/clf4_2022-02-15_ARG']

In [6]:
import numpy as np
from pathlib import Path

qstrings = np.unique([Path(f).name.split('_')[-2] for f in  FilesActual])
ystrings = np.unique([Path(f).name.split('_')[-2].split('-')[0] for f in  FilesActual])
yrs = [int(y) for y in ystrings]

startyr = min(yrs)
endyr = max(yrs) + 1

In [7]:
import os

if not os.path.exists('./../../samplerCensoARG/'):
    !git clone https://github.com/matuteiglesias/samplerCensoARG.git ./../../

In [8]:
frac = 0.02 ## Frac needs to be the fraction used in the sampling (eg. -f 0.01 needs frac = 0.01)

sample_tag = 'CSactual'

# Comentar si los datasets ya estan calculados
# for j in range(2):
#     sample_tag = str(10*j).zfill(3)
#     print(sample_tag)
    
# !python ./../../samplerCensoARG/samplear.py -dbp '/media/miglesia/Elements/suite/ext_CPV2010_basico_radio_pub' -f $frac -y $startyr $endyr -n $sample_tag


In [8]:

AGLO_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/AGLO_rk')
rk_table = AGLO_rk.set_index(['ANO4', 'AGLOMERADO']).unstack()
AGLO_rk_filled = rk_table.fillna(rk_table.mean()).stack().reset_index()
AGLO_rk = AGLO_rk_filled

Reg_rk = pd.read_csv('./../../encuestador-de-hogares/data/info/Reg_rk')

dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')

# for j in range(2):
#     sample_tag = str(10*j).zfill(3)
#     print(sample_tag)
    
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    out_filename = '/media/miglesia/Elements/suite/yr_samples/table_f'+str(frac)+'_'+yr+'_'+sample_tag+'.csv'

    if not os.exists(out_filename):

        table = pd.read_csv('./../../samplerCensoARG/data/censo_samples/table_f'+str(frac)+'_'+yr+'_'+sample_tag+'.csv')
        table['ANO4'] = int(yr)

        # Adaptamos las categorias de respuestas para que iguales las de la EPH
        ## VIVIENDA
        table['V01'] = table['V01'].map({1:1, 2:6, 3:6, 4:2, 5:3, 6:4, 7:5, 8:6})
        ## HOGAR
        table['H06'] = table['H06'].map({1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:9})
        table['H09'] = table['H09'].map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4})
        table['H16'] = table['H16'].clip(0, 9)
        table['H14'] = table['H14'].map({1:1, 2:4, 3:2, 4:2, 5:4, 6:3, 7:4, 8:9})
        table['H13'] = table['H13'].map({1:1, 2:2, 4:0})
        # PERSONA
        table['P07'] = table['P07'].map({1:1, 2:2, 0:2})

        ## Agregar Region
        table = table.merge(dpto_region[['DPTO', 'Region']])

        ## Agregar ranking de Region y Aglo
        print(table.shape)
        table = table.merge(AGLO_rk[['AGLOMERADO', 'ANO4', 'AGLO_rk']]).merge(Reg_rk[['Region', 'ANO4', 'Reg_rk']])
        print(table.shape)

        table.to_csv(out_filename, index = False)  # Copias en carpeta yr_samples, en nuestra carpeta de indice de pobreza


SyntaxError: invalid syntax (<ipython-input-8-42b8a69ce876>, line 18)

# 2. Predecir  (X --> y)

## Info Empleo

In [9]:
empleo = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/empleoARG/main/datos/45.2_ECTDT.csv')
empleo = empleo[['45.2_IT_0_T_13', '45.2_ECTDT_0_T_33']] # ('45.2_ECTDT_0_T_33' es tasa de desocupacion en total aglomerados)
empleo['Q'] = pd.to_datetime(empleo['45.2_IT_0_T_13']) + pd.DateOffset(months=1, days = 14)
empleo = empleo.set_index('Q').drop(['45.2_IT_0_T_13'], axis = 1)
empleo = empleo.replace('s/d', np.nan).astype(float).round(4)
empleo['censo2010_ratio'] = (empleo/empleo.loc['2010-11-15'])

## notar que la tasa en Aglos, segun el censo, no es igual al valor de la serie de tiempo.
# para oct 2010 el censo da (6.29 %) y la que tenemos en dato (7.5%)
desoc_C2010 = pd.read_csv('./../data/info/desoc_AGLOsi_C2010.csv')
tasa_C2010 = desoc_C2010.loc[desoc_C2010.AGLO_si == True]['Tasa desocupacion'].values[0]
tasa_C2010

0.0628729377307203

In [8]:
import json

### Funcion ajustar nivel de empleo

def ajustar_empleo(data, verbose = False):

        ratio = empleo.loc[pd.to_datetime(q)].censo2010_ratio
        n_desempleados_ = ratio*(CONDACT_cnts[1] + CONDACT_cnts[2])*tasa_C2010
        desemp_adic = round(n_desempleados_ - CONDACT_cnts.loc[2]) # Desempleados adicionales
        
        print(str(q)[:10])

        if desemp_adic > 0:
            data.loc[
                data.query('CONDACT == 1').sample(desemp_adic).index,
                'CONDACT'
            ] = 2
        elif desemp_adic < 0:
            data.loc[
                data.query('CONDACT == 2').sample(- desemp_adic).index,
                'CONDACT'
            ] = 1

        if verbose:
            desempleo = data.CONDACT.value_counts().loc[2] / (data.CONDACT.value_counts().loc[1] + data.CONDACT.value_counts().loc[2])
            print('desempleo:' + str(desempleo))
        
        return data
    
    
def predict_save(X_data, x_cols, y_cols, model_filename, out_filename, balance_proba, tag, overwrite = False):

        # Si todavia no existe la training data de ese anio, o si la opcion overwrite esta activada:
        if (not os.path.exists(out_filename)) or (overwrite): 

            CLF = joblib.load(model_filename)
            
            if balance_proba: ## Prediccion usando predict proba y factores de balance.
                y_out = pd.DataFrame([])
                proba_values = CLF.predict_proba(X_data[x_cols])

                ## Leer factores de archivo json
                with open('/home/miglesia/repositories/encuestador-de-hogares/data/training/factors/'+tag+'.json', 'r') as file:
                    info = json.load(file)
                factors_mean = pd.DataFrame(json.loads(info[tag])).stack()
                factors_mean.index.names = ['variable', 'valor']
                factors_mean.index = factors_mean.index.set_levels(factors_mean.index.levels[1].astype(float).astype(int), level='valor')

                for j, y_col in enumerate(y_cols):

                    y_probas = pd.DataFrame(proba_values[j], columns = factors_mean.loc[y_col].index.values)
                    y_out[y_col] = (y_probas/factors_mean.loc[y_col]).idxmax(1)#.value_counts().sort_index()
                
            else: ## Prediccion sin balance
                y_out = CLF.predict(X_data[x_cols].values)

            ## Listo
            y_censo_fit = pd.DataFrame(y_out, index = X_data.index, columns=y_cols)
            
            Xy_censo = pd.concat([X_data, y_censo_fit], axis = 1)

#             save
            Xy_censo.to_csv(out_filename, index = False)
            print('File saved at '+ out_filename)
            del X_data; del Xy_censo; del CLF
#             gc.collect()

In [9]:
overwrite = False

## Elegir el dataset usado como X:
frac = '0.02'

# models_path = '/media/miglesia/Elements/suite/estocasticidad_indice_pobreza'
models_path = '/media/miglesia/Elements/suite/encuestador-de-hogares'

balance_proba = False

import joblib

models_tag = 'ARG'
sample_tag = 'CSactual'

# for i in range(n_models):
#     models_tag = str(i).zfill(3)

#     for j in range(2):
#         sample_tag = str(10*j).zfill(3)
experiment_tag = models_tag + sample_tag

print(experiment_tag)

ARGCSactual


In [12]:


for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    file_ = '/media/miglesia/Elements/suite/yr_samples/table_f'+str(frac)+'_'+yr+'_'+sample_tag+'.csv'

    X_censo = pd.read_csv(file_, usecols = ['DPTO','RADIO_REF_ID','PERSONA_REF_ID', 'HOGAR_REF_ID','IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'URP', 'V01', 'H05', 'H06',
           'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14', 'AGLO_rk', 'Reg_rk',
           'H13', 'P07', 'P08', 'P09', 'P10', 'P05']).fillna(0)

    ## Tratamiento trimestral 
#             qs = np.array(allqs)[[i for i, si in enumerate(allqs) if si.startswith(yr)]]
    qs = np.array(qstrings)[[i for i, si in enumerate(qstrings) if si.startswith(yr)]]
    print(qs)

    CONDACT_cnts = X_censo.CONDACT.value_counts()

#             print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])

    ### Cargar modelos de la parte no trimestral (anual).
    for q in sorted(qs):

        ### AJUSTAR NIVEL DE DESEMPLEO
        X_q = X_censo.copy()
        X_q['Q'] = q
        print('Nuevo trimestre.')

        X_q = ajustar_empleo(X_q)

        #################################    #################################    #################################

        print('C1')
        ## CLASIF 1
        X_data = X_q;
        y_cols1 = ['CAT_OCUP', 'CAT_INAC', 'CH07']
        x_cols1 = ['IX_TOT', 'P02', 'P03', 'AGLO_rk', 'Reg_rk', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05', 'CONDACT']
        out_filename1 = '/media/miglesia/Elements/suite/yr_samples/RFC1_'+str(frac)+'_'+str(q)[:10]+'_'+experiment_tag+'.csv'

        predict_save(X_data,
                     x_cols = x_cols1,
                     y_cols = y_cols1,
                     out_filename = out_filename1,
                     model_filename = models_path + '/fitted_RF/clf1_'+yr+'_'+models_tag,
                     balance_proba = balance_proba,
                     tag = 'clf1_'+yr+'_'+models_tag,
                    overwrite = overwrite)

        del X_q; del X_data
#         print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])

        #################################    #################################    #################################

        print('C2')
        ## CLASIF 2
        X_data = pd.read_csv(out_filename1)
        y_cols2 = ['INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS']
        x_cols2 = x_cols1 + y_cols1
        out_filename2 = '/media/miglesia/Elements/suite/yr_samples/RFC2_'+str(frac)+'_'+str(q)[:10]+'_'+experiment_tag+'.csv'

        predict_save(X_data,
                     x_cols = x_cols2,
                     y_cols = y_cols2,
                     out_filename = out_filename2,
                     model_filename = models_path + '/fitted_RF/clf2_'+yr+'_'+models_tag,
                     balance_proba = balance_proba,
                     tag = 'clf2_'+yr+'_'+models_tag,
                    overwrite = overwrite)

        del X_data
#         print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])

        #################################    #################################    #################################

        print('C3')

        ## CLASIF 3
        X_data = pd.read_csv(out_filename2)
        y_cols3 = ['PP07G1','PP07G_59', 'PP07I', 'PP07J', 'PP07K']
        x_cols3 = x_cols2 + y_cols2
        out_filename3 = '/media/miglesia/Elements/suite/yr_samples/RFC3_'+str(frac)+'_'+str(q)[:10]+'_'+experiment_tag+'.csv'

        predict_save(X_data,
                     x_cols = x_cols3,
                     y_cols = y_cols3,
                     out_filename = out_filename3,
                     model_filename = models_path + '/fitted_RF/clf3_'+yr+'_'+models_tag,
                     balance_proba = balance_proba,
                     tag = 'clf3_'+yr+'_'+models_tag,
                    overwrite = overwrite)
        del X_data
#         print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])

        #################################    #################################    #################################


        # Columnas de ingresos. Necesitan una regresion...
        columnas_pesos = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']

        x_cols4 = x_cols3 + y_cols3
        # Columnas de ingresos. Necesitan una regresion...
        predecir4 = columnas_pesos
        y_cols4 = predecir4


        print('reg')
        # REGRESION            
        out_filename4 = '/media/miglesia/Elements/suite/yr_samples/RFReg_'+str(frac)+'_'+str(q)[:10]+'_'+experiment_tag+'.csv'
        if (not os.path.exists(out_filename4)) or (overwrite): 

            ## Cargar Modelo
            model_filename4 = models_path + '/fitted_RF/clf4_'+str(q)[:10]+'_'+models_tag
    #         filename = '/media/miglesia/Elements/CENSO_dirs/Pobreza/fitted_RF/clf4_0.02_'+str(q)[:10]+'_'+experiment_tag+'.sav'
#             clf4 = pickle.load(open(model_filename4+'sav', 'rb'))
            clf4 = joblib.load(model_filename4)

            Xy3_censo = pd.read_csv(out_filename3)
            y_out4 = clf4.predict(Xy3_censo[x_cols4].values); del clf4
            y_censo_fit4 = pd.DataFrame(y_out4, index = Xy3_censo.index, columns=predecir4)

            Xy4_censo = pd.concat([Xy3_censo, y_censo_fit4], axis = 1)
#             save
            Xy4_censo.to_csv(out_filename4, index = False)

            del Xy4_censo;

    del X_censo; #del clf1; del clf2; del clf3


ARGCSactual
2021
['2021-05-15' '2021-08-15' '2021-11-15' '2022-02-15']
Nuevo trimestre.
2021-05-15
C1
C2
C3
reg
Nuevo trimestre.
2021-08-15
C1
C2
C3
reg
Nuevo trimestre.
2021-11-15
C1
C2
C3
reg
Nuevo trimestre.
2022-02-15
C1
C2
C3
reg
2022
['2021-05-15' '2021-08-15' '2021-11-15' '2022-02-15']
Nuevo trimestre.
2021-05-15
C1
C2
C3
reg
Nuevo trimestre.
2021-08-15
C1
C2
C3
reg
Nuevo trimestre.
2021-11-15
C1
C2
C3
reg
Nuevo trimestre.
2022-02-15
C1
C2
C3
reg


# 3. Computar Pobreza

In [10]:
# Adulto equivalente. Cuanto cuesta la manutencion de las personas segun sexo y edad.
ad_eq = pd.read_csv('./../data/info/adulto_eq.csv')

#Importar canasta basica regional deflac
CB_ipc = pd.read_csv('./../data/info/CB_Reg_defl.csv')
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')#.merge(aglo_labels)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')

## Fix temporario, migracion a nombres de region oficiales
dpto_region['Region'] = dpto_region['Region'].map({'Gran Buenos Aires':'gran_buenos_aires', 
                                                   'Pampeana':'pampeana', 'Noroeste':'noroeste', 
                                                   'Noreste':'noreste','Patagónica': 'patagonia', 'Cuyo': 'cuyo'})

radio_ref = radio_ref.merge(dpto_region)
# dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
DPTO_Region = radio_ref[['DPTO', 'Region']].drop_duplicates()

In [11]:
import os
if not os.path.exists('./../data/Pobreza/'):
    os.makedirs('./../data/Pobreza/')

frac = 0.02

In [12]:

def ingresos_a_pobreza(df_ingresos, filename, columnas_pesos = ['P47T']):

    df_ingresos[columnas_pesos] = np.power(10, df_ingresos[columnas_pesos]) - 1

    df = df_ingresos.reset_index()

    ## CANASTA: Datos mergeado con adulto equivalente, region y serie de tiempo canasta
    df_cb = df_ingresos.merge(ad_eq).merge(DPTO_Region).merge(CB_ipc)#.merge(ppp_defl[['Q', 'ppp_5usd_ARS_deflac']])
    df_cb['CBA'] = df_cb['CBA']*df_cb['CB_EQUIV']  ## Con este paso el valor de canasta de una persona YA INCORPORA EL AD EQ
    df_cb['CBT'] = df_cb['CBT']*df_cb['CB_EQUIV']  ## Con este paso el valor de canasta de una persona YA INCORPORA EL AD EQ

    ## VARIABLES A NIVEL HOGARES
#     df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT', 'CB_EQUIV', 'ppp_5usd_ARS_deflac']].sum()
    df_cb_hogares = df_cb.groupby(['HOGAR_REF_ID', 'Q'])[['P47T','CBA', 'CBT', 'CB_EQUIV']].sum()
    df_cb_hogares['Pobreza'] = df_cb_hogares['P47T'] < df_cb_hogares['CBT']
    df_cb_hogares['Indigencia'] = df_cb_hogares['P47T'] < df_cb_hogares['CBA']
#     df_cb_hogares['Pobreza_5usd'] = df_cb_hogares['P47T'] < df_cb_hogares['ppp_5usd_ARS_deflac']
#     pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT', 'CB_EQUIV','Pobreza', 'Indigencia', 'Pobreza_5usd']].reset_index()
    pobreza_hogares = df_cb_hogares[['P47T','CBA','CBT', 'CB_EQUIV','Pobreza', 'Indigencia']].reset_index()
    pobreza_hogares['gap_pobreza'] = pobreza_hogares.P47T - pobreza_hogares.CBT
    pobreza_hogares['gap_indigencia'] = pobreza_hogares.P47T - pobreza_hogares.CBA
    pobreza_hogares = pobreza_hogares.rename(columns = {'P47T': 'P47T_hogar'})

    ## UNION DE DATOS DE HOGARES A REGISTROS INDIVIDUALES
    data = df_ingresos.merge(pobreza_hogares, on = ['HOGAR_REF_ID', 'Q'])#, how = 'left')
    del df; del pobreza_hogares # Ahorrar memoria
    data = data.rename(columns = {'P47T': 'P47T_persona'}) # Renombrar la variable P47T para aclarar que es a nivel persona.

    ## UNIR INFO GEOGRAFICA
    data = data.merge(radio_ref[['RADIO_REF_ID', 'IDFRAC', 'PROV', 'NOMPROV', 'AGLOMERADO', 'Region']].drop_duplicates())

    n_q = data.Q.nunique()
    print("Poblacion: "+str(len(data)/frac/n_q))
#     display(data[['Pobreza', 'Indigencia', 'Pobreza_5usd']].mean())
    data.to_csv(filename, index = False) ## Aca si ya existen no deberian sobreescribirse (o si)
    
    print(filename+' saved')

In [13]:
Xy_trimestrales = '/media/miglesia/Elements/suite/yr_samples/RFReg_'+str(frac)  # use your path

allFiles = []
allFiles += glob.glob(Xy_trimestrales +'*'+experiment_tag+'.csv')
allFiles = sorted(allFiles)
CSXyfiles = allFiles[-4:]
CSXyfiles

['/media/miglesia/Elements/suite/yr_samples/RFReg_0.02_2021-05-15_ARGCSactual.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.02_2021-08-15_ARGCSactual.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.02_2021-11-15_ARGCSactual.csv',
 '/media/miglesia/Elements/suite/yr_samples/RFReg_0.02_2022-02-15_ARGCSactual.csv']

In [14]:

print(experiment_tag)
filename = '/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_'+'_'.join([str(frac), experiment_tag])+'.csv'

df_parts = []
for quarter_Xy_file in sorted(CSXyfiles):# ultimo anio
    df_Q = pd.read_csv(quarter_Xy_file, 
                           usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID','RADIO_REF_ID', 'CONDACT', 'CAT_INAC', 'CAT_OCUP',
                                      'INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS', 'PP07K',
                                      'IX_TOT', 'H16', 'H15','P47T', 'P03','P02', 'P09','P10', 'DPTO', 'URP'])

    df_Q['ANO4'] = int(Path(quarter_Xy_file).name.split('_')[-2].split('-')[0])
    q = Path(quarter_Xy_file).name.split('_')[-2]; print(q)
    df_Q['Q'] = q
    df_parts += [df_Q]

df = pd.concat(df_parts)
del df_Q


ingresos_a_pobreza(df_ingresos = df, filename = filename, columnas_pesos = ['P47T'])

ARGCSactual
2021-05-15
2021-08-15
2021-11-15
2022-02-15
Poblacion: 44435950.0
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.02_ARGCSactual.csv saved


In [ ]:
xx

# Up to here. The rest of the code is on another notebook.

## Los niveles geograficos disponibles son:
 - Radios (RADIO_REF_ID)
 - Fracciones (IDFRAC, no es clave unica)
 - Dptos (DPTO)
 - Provs (PROV)
 - Aglos (AGLOMERADO)
 - (Region)

In [ ]:
>>> import seaborn as sns
>>> pal = sns.color_palette("RdYlGn_r", 10)
>>> print(pal.as_hex())


In [ ]:
>>> import seaborn as sns
>>> pal = sns.color_palette("cool", 10)
>>> print(pal.as_hex())